# Memory

## Outline

In previous version, the following memory classes were used:

* **ConversationBufferMemory:** allows for storing of messages and then extract them in a variable.
* **ConversationBufferWindowMemory:** keeps a list of the interaction of the conversation over tim. It uses the last K interactions.
* **ConversationTokenBufferMemory:** keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush.
* **ConversationSummaryMemory:** creates a summary of the conversation over time.

Also it supports vector and entity memory and can use multiple memories like Conversational + Entity (to recall individuals).

However, they were deprecated so it is recommended to use either BaseChatMessageHistory or RunnableWithMessageHistory.
Also follow the recommended memory migration guide https://python.langchain.com/docs/versions/migrating_memory/

## RunnableWithMessageHistory

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

store = {}  # memory is maintained outside the chain
session_id = "1" # example session ID, can be any unique identifier
chat_thread_config = {
    "configurable": { "session_id": session_id }
}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

llm = ChatOllama(model='llama3.2:1b', temperature=0.0)

chain = RunnableWithMessageHistory(llm, get_session_history)

Let's start a conversation with the chain and ask a question about the movies mentioned in the conversation.

In [2]:
chain.invoke("Hi I'm Rick. I have watched 'Inception' and 'The Matrix'.", config=chat_thread_config)
store[session_id].add_user_message("Also, I have watched 'Interstellar'.")
question_response = chain.invoke("Can you tell me the name of the movies I have mentioned before in this conversation?", config=chat_thread_config)
print(question_response.content)

You've also watched "Interstellar", Rick. That's a great film as well, exploring complex themes like time dilation, wormholes, and the search for a new home for humanity.

As for your previous mentions, you've listed:

1. "Inception"
2. "The Matrix"
3. "Interstellar"

Let me know if you'd like to discuss any of these films further or if you have any other questions!


`session_id` helps to differentiate between chats so if providing a different id, the AI won't answer with the movies

In [3]:
question_response = chain.invoke(
  "Can you tell me the name of the two movies I have mentioned before in this conversation?",
  config={ "configurable": { "session_id": "2" } }
)
print(question_response.content)

This is the beginning of our conversation, and we haven't discussed any movies yet. What would you like to talk about or ask? I can try to help you remember some movie titles if you'd like.
